In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [2]:
func_out = {'sigmoid': sigmoid_t,'tanh': tanh_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':2}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results/results_gvqs_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time','RSS_h'], axis =1 )
        tmp = []
        for el in df["in_train_weights"]:
             tmp.append(el[-1])
        df["weights"] = tmp
        df = df.drop(["in_train_weights"], axis = 1)
        tot[f"{flab}{nq}"] = df 

In [3]:
rss=[]
for flab, fun in func_out.items():
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        for i, el in dl.iterrows():
            
            lower = 0.
            upper = 1.

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
            y = [fun(value,f_i) for value in xx]

            #tck=splrep(x,y,k=1) #coeffs
            norm = np.linalg.norm(y)
            y = y / norm
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
            rmse_scal.append(math.sqrt(np.square(np.subtract(y,y_fq)).mean()))
        dl["rmse_scal"] = rmse_scal
        dl["rmse_yc"] = rmse_yc


In [4]:
tot.keys()

dict_keys(['sigmoid3', 'sigmoid4', 'tanh3', 'tanh4', 'elu3', 'elu4', 'relu3', 'relu4', 'sin3', 'sin4'])

In [5]:
dk= tot["sigmoid3"]
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
print(dk.pivot( columns = [], values=['rmse_yc','RSS_q']).sort_values(["rmse_yc"]).to_markdown())

|       |   rmse_yc |    RSS_q |
|:------|----------:|---------:|
| (12,) |  0.183223 | 0.268566 |
| (22,) |  0.18553  | 0.275371 |
| (11,) |  0.185691 | 0.275849 |
| (3,)  |  0.185943 | 0.276599 |
| (23,) |  0.698045 | 3.89813  |
| (6,)  |  0.702102 | 3.94358  |
| (17,) |  0.702309 | 3.9459   |
| (13,) |  0.702592 | 3.94908  |
| (18,) |  0.703671 | 3.96122  |
| (0,)  |  0.703877 | 3.96354  |
| (8,)  |  0.704963 | 3.97578  |
| (10,) |  0.706013 | 3.98764  |
| (9,)  |  0.715782 | 4.09875  |
| (19,) |  0.719192 | 4.1379   |
| (4,)  |  0.719874 | 4.14575  |
| (20,) |  0.719874 | 4.14575  |
| (14,) |  0.721589 | 4.16553  |
| (15,) |  0.725029 | 4.20533  |
| (7,)  |  0.738071 | 4.35799  |
| (5,)  |  1.08453  | 9.40971  |
| (21,) |  1.08523  | 9.42185  |
| (2,)  |  1.08552  | 9.42683  |
| (1,)  |  1.08588  | 9.43312  |
| (16,) |  1.08667  | 9.44684  |
| (24,) |  1.08667  | 9.44684  |


In [6]:
flab = "sigmoid"
nq = 3
dl = tot[f"{flab}{nq}"]
dl["RSS_q"][3]


0.2765988787

In [23]:
rss=[]
res = {}

for flab, fun in func_out.items():
    funres = {}
    # vqs results
    for stp in [20,40]:
        nq = 1
        file_path = f"results/results_vqs_{flab}_{nq}_{stp}.json"
        dl = pd.read_json(file_path).drop(["exe_time","weights", "training_cost", "seed"], axis = 1)

        funres[f"vqs_{stp}"]= {"rmse":np.min(dl["rmse"]), "index": np.argmin(dl["rmse"]), "RSS": dl["RSS_q"][np.argmin(dl["rmse"])] }


    # gvqs results
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        lower = 0.
        upper = 1.
        for i, el in dl.iterrows():
            

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1

            #tck=splrep(x,y,k=1) #coeffs
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) /np.linalg.norm(vector)#classic
            
            y_fq=[]
            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(v_norm,np.abs(y_fq))).mean()))
        funres[f"gvqs_{nq}"]= {"rmse":np.min(rmse_yc), "index": np.argmin(rmse_yc), "RSS": dl["RSS_q"][np.argmin(rmse_yc)] }
    res[flab] = funres


In [26]:
for key in res.keys():
    print('\n' + '='*15 + '  ' +key+ '  ' + '='*15)
    print(pd.DataFrame(res[key]).T.to_markdown(tablefmt= 'latex', floatfmt=".4f"))



===============  sigmoid  ===============
\begin{tabular}{lrrr}
\hline
        &   rmse &   index &     RSS \\
\hline
 vqs\_20 & 0.0868 &  3.0000 &  0.3011 \\
 vqs\_40 & 0.0872 &  5.0000 &  0.6086 \\
 gvqs\_3 & 0.1212 & 16.0000 &  9.4468 \\
 gvqs\_4 & 0.0753 &  3.0000 & 14.0297 \\
\hline
\end{tabular}

===============  tanh  ===============
\begin{tabular}{lrrr}
\hline
        &   rmse &   index &     RSS \\
\hline
 vqs\_20 & 0.2135 & 17.0000 &  1.8230 \\
 vqs\_40 & 0.2042 &  6.0000 &  3.3360 \\
 gvqs\_3 & 0.1184 & 10.0000 &  8.4393 \\
 gvqs\_4 & 0.0729 &  2.0000 & 12.3363 \\
\hline
\end{tabular}

===============  elu  ===============
\begin{tabular}{lrrr}
\hline
        &   rmse &   index &    RSS \\
\hline
 vqs\_20 & 0.1543 &  1.0000 & 0.9519 \\
 vqs\_40 & 0.1582 &  4.0000 & 2.0013 \\
 gvqs\_3 & 0.1399 &  4.0000 & 5.7686 \\
 gvqs\_4 & 0.0774 &  9.0000 & 0.0722 \\
\hline
\end{tabular}

===============  relu  ===============
\begin{tabular}{lrrr}
\hline
        &   rmse &   index &   